In [19]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import (mean_squared_error, plot_confusion_matrix, plot_roc_curve, 
                             classification_report, precision_recall_curve, confusion_matrix)
import statsmodels.api as sm
from pathlib import Path
import numpy as np
import seaborn as sns

In [20]:
folder = "data/comp1_2020_1/"
Path(folder).mkdir(parents=True, exist_ok=True)

In [21]:
with open(f'{folder}/dropout_df_total_and_per_week_dict_3_6.pkl', 'rb') as pklfile:
    df_total_and_per_week = pickle.load(pklfile)

In [22]:
exclude = df_total_and_per_week[(df_total_and_per_week['attempts_12'] == 0) & \
                                (df_total_and_per_week['attempts_13'] == 0) & \
                                (df_total_and_per_week['attempts_14'] == 0) & \
                                (df_total_and_per_week['attempts_15'] == 0)]
dtw2 = df_total_and_per_week.drop(index=exclude.index)
dtw2

,attempts_12,success_rate_12,avg_timedelta_to_solution_12,avg_timedelta_from_beginning_12,on_time_exercises_12,frequency_12,success_count_12,attempts_13,success_rate_13,avg_timedelta_to_solution_13,...,on_time_exercises_19,frequency_19,success_count_19,attempts,success_rate,avg_timedelta_to_solution,avg_timedelta_from_beginning,on_time_exercises,frequency,success_count
910,8,0.375000,65.386857,328.720191,3,1,3,24,0.291667,291.476823,...,6,1,6,32,0.312500,223.649833,519.849833,10,2,10
911,4,0.750000,132.823499,684.156833,3,1,3,36,0.194444,23916.274449,...,6,2,6,40,0.250000,16781.239164,17381.439164,10,3,10
912,6,0.500000,24.270101,207.270101,3,1,3,7,1.000000,0.000000,...,6,1,6,13,0.769231,7.281030,141.581030,10,2,10
914,9,0.333333,430.621847,918.621847,3,1,3,91,0.076923,40044.526822,...,6,2,6,100,0.100000,28160.355329,28983.855329,10,4,10
915,8,0.375000,124.739469,243.072802,3,1,3,12,0.416667,18.759911,...,5,1,5,20,0.400000,58.502245,167.252245,8,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,0,NaN,NaN,NaN,0,3,0,0,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,0,3,0
1403,0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,...,5,1,5,0,NaN,NaN,NaN,0,0,0
1405,6,0.500000,32.428669,655.095336,0,2,3,22,0.181818,26859.988079,...,0,0,0,28,0.250000,15362.462618,15919.319761,0,7,7
1407,56,0.053571,2124.487458,3170.154125,0,1,3,0,NaN,NaN,...,0,0,0,56,0.053571,NaN,NaN,0,3,3


In [23]:
dtw2['success_count_label'] = dtw2['success_count_17'] + dtw2['success_count_19']

In [24]:
# Features and fill missing data
features_week6 = dtw2[['on_time_exercises_12',
                       'on_time_exercises_13',
                       'on_time_exercises_14',
                       'on_time_exercises_15',
                      ]].reset_index(drop=True)


In [25]:
features_week6.head()

,on_time_exercises_12,on_time_exercises_13,on_time_exercises_14,on_time_exercises_15
0,3,7,7,6
1,3,7,7,6
2,3,7,7,6
3,3,7,7,6
4,3,5,6,6


In [26]:
# Target
outcome_week6 = dtw2['success_count_label'].reset_index(drop=True)

In [27]:
outcome_week6

0      12
1      12
2      12
3      12
4      11
       ..
405     0
406    11
407     0
408     0
409     0
Name: success_count_label, Length: 410, dtype: int64

In [28]:
X_train, X_test, y_train, y_test = train_test_split(features_week6, outcome_week6, test_size=0.2, random_state=0)
idx = np.array(X_train.index.tolist())

In [29]:
df = dtw2[['on_time_exercises_12',
                       'on_time_exercises_13',
                       'on_time_exercises_14',
                       'success_count_label']]

In [30]:
X_train_const = sm.add_constant(X_train)
poisson_model = sm.GLM(y_train, X_train, family=sm.families.Poisson())

In [31]:
y_pred = poisson_model.predict(X_test)

ValueError: shapes (328,4) and (82,4) not aligned: 4 (dim 1) != 82 (dim 0)

In [ ]:
y_pred = poisson_model.predict(X_test_const)

ValueError: shapes (328,4) and (82,4) not aligned: 4 (dim 1) != 82 (dim 0)

In [ ]:
y_pred_PR

array([ 7.92389363,  5.0469288 , 11.68097717,  6.60269358, 12.07071692,
       12.07071692,  5.6868759 , 12.07071692,  9.7359401 ,  8.74380185,
        9.42158571,  6.02650122,  7.19969048,  7.90700994,  9.5930307 ,
        6.35044866,  5.0469288 ,  5.0469288 ,  8.95204115, 11.43916363,
       10.36651074, 12.07071692,  9.39444075,  5.0469288 , 12.07071692,
        8.01876755,  6.25723333, 12.07071692, 12.07071692, 11.43916363,
        6.02650122, 11.06981549, 11.06981549,  8.71860969,  6.58543221,
        9.62074948, 12.07071692,  5.0469288 ,  5.75496569,  8.01876755,
        6.58543221,  5.0469288 ,  9.25072903, 12.07071692,  6.71536394,
       10.93884314, 11.04622869, 12.07071692, 12.07071692,  5.0469288 ,
       12.07071692,  5.0469288 ,  5.0469288 ,  7.35188541, 12.07071692,
        5.0469288 ,  5.61959172, 10.68956765,  9.1173812 ,  6.58543221,
        8.82299886,  6.14540513,  6.58543221,  9.25072903, 12.07071692,
       12.07071692,  5.0469288 ,  6.87196294, 10.71239288,  7.65

In [ ]:
rmse = mean_squared_error(y_test, y_pred_PR, squared=False)
print("RMSE: %.2f" % rmse)

RMSE: 3.86


In [96]:
len(y_test)

82

In [102]:
from cmath import nan

classifiers = list()
for c in np.arange(0,12.1,0.1):
    for k in [4,5,6,7]:
        predictions = list()
        for i in range(len(y_pred_PR)):
            if y_pred_PR[i] < c and y_test[i] < k:
                pred = "VP"
            elif y_pred_PR[i] < c and y_test[i] > k:
                pred = "FP"
            elif y_pred_PR[i] > c and y_test[i] > k:
                pred = "VN"
            else:
                pred = "FN"
            predictions.append(pred)
        
        VP = predictions.count("VP")
        VN = predictions.count("VN")
        FP = predictions.count("FP")
        FN = predictions.count("FN")
        TFP = FP/(FP+VN)
        TVP = VP/(VP+FN)
        TVN = VN/(FP+VN)
        try:
            VPP = VP/(VP+FP)
            F1_P = 2*VPP*TVP/(VPP+TVP)
        except:
            VPP = nan
            F1_P = nan
        try:
            VPN = VN/(FN+VN)
            F1_N = 2*VPN*TVN/(VPN+TVN)
        except:
            VPN = nan
            F1_N = nan

        classifiers.append({"C": c,
                            "K": str(k),
                            "VP": VP,
                            "VN": VN,
                            "FP": FP,
                            "FN": FN,
                            "TFP": TFP,
                            "TVP": TVP,
                            "TVN": TVN,
                            "F1_P": F1_P,
                            "F1_N": F1_N})
classifiers = pd.DataFrame(classifiers)

KeyError: 0